In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
from konlpy.tag import Okt as Twitter
import gensim 
import tensorflow as tf
import numpy as np
import codecs
from pprint import pprint
import nltk
import re

In [18]:
#1. dir에 있는 txt데이터 읽기
def read_data(filename):    
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]        
        data = data[1:]   # header 제외 #    
    return data

train_data = list()
for (path, dir, files) in os.walk(os.getcwd()+'/trans2/google'):
    
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            
            train_data.append(read_data(path+'/'+filename))

# print(len(train_data))      # nrows: 1701
# print(len(train_data[0]))   # ncols: 4
# print(train_data[0][1])

#2. shape4가 다른 녀석들 읽지말고
def del_anomaly_shape(data):
    a = []
#     if shape == 3:
#         for i in range(len(data)):
#     #         data size 3가 되야 정상
#             if len(data[i]) != 3:
#                 a.append(i)
    for i in range(len(data)):
    # data size 4가 되야 정상
        if len(data[i]) != 4:
            a.append(i)

            
    for index in sorted(a, reverse=True):
        del data[index]
#     print(len(a))
    
    return data


# 3. data[1]인녀석들의 영어와 숫자를 모두 제거한다. 
def preprocess(text):
    data = list()
    for line in text:
        sentence = re.sub(r"[a-zA-Z0-9]+", '', line)
        data.append(sentence)
    
    retrun data
    


def tokenize(doc):

    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]






# 형태소로 토크나이징
pos_tagger = Twitter()

tmp = list()
for i in range(len(train_data)):
    tmp.append(del_anomaly_shape(train_data[i]))

# 전처리
train_data = preprocess(tmp)
train_docs= list()
    
# train_data2 = del_anomaly_shape(train_data2)
# # test_data = del_anomaly_shape(test_data)

# # training Word2Vec model using skip-gram   
# # row[0][3][0] = 분노의 정도 (int값)


for i in train_data:
    train_docs += [tokenize(row[1]) for row in i]

# train_docs2 = [(tokenize(row[1]),row[3][0]) for row in train_data2]
# # test_docs = [(tokenize(row[1]), row[3][0]) for row in test_data]

tokens = [t for d in train_docs for t in d]

# print(len(tokens))



# # #########  Nltk data exploration
# text = nltk.Text(tokens, name='NMSC')
# # 80만개 선택
# selected_words = [f[0] for f in text.vocab().most_common(800000)]
# text = selected_words


model = gensim.models.Word2Vec(size=300, window=2 ,workers=4, min_count=50,sg = 1, alpha=0.025,min_alpha=0.025, seed=1234)

model.build_vocab(tokens)


for epoch in range(30):
           
    model.train(tokens,total_examples=model.corpus_count,epochs = model.iter)
    model.alpha -= 0.002
    model.min_alpha = model.alpha
    print(epoch)
    
    
model.save('./Word2vec.model')
# model.most_similar('분노/noun',topn = 20)  ## topn = len(model.wv.vocab)

32
1105
['2018-Ko-03093', '내가 따라야 할 좋은 #funny #entertaining # 관심있는 계정은 무엇입니까? 내 트위터가 건조한', '기쁨', '1 : 낮은 기쁨을 유추 할 수 있음']


/home/minwookje/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel/__main__.py:104: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [1]:
import gensim
import tensorflow as tf
import codecs
import os
import numpy as np

model = gensim.models.word2vec.Word2Vec.load('Word2vec.model')
max_size = len(model.wv.vocab)-1
w2v = np.zeros((max_size,model.layer1_size))

# metadata.tsv업데이트하기
with codecs.open("metadata.tsv",'w+',encoding='utf8') as file_metadata:
    for i,word in enumerate(model.wv.index2word[:max_size]):
        w2v[i] = model.wv[word]
        file_metadata.write(word + "\n")

# from tensorflow.contrib.tensorboard.plugins import projector


# sess = tf.InteractiveSession()
# ##  Create 2D tensor called embedding that holds our embeddings ##  
# with tf.device("/cpu:0"):
#     embedding = tf.Variable(w2v, trainable = False,  name = 'embedding')   

# tf.global_variables_initializer().run()

# path = 'word2vec'

# saver = tf.train.Saver()
# writer = tf.summary.FileWriter(path, sess.graph)

# ## adding into project
# config = projector.ProjectorConfig()
# embed = config.embeddings.add()
# embed.tensor_name = 'embedding'
# embed.metadata_path = './metadata.tsv'
# # Specify the width and height of a single thumbnail.
# projector.visualize_embeddings(writer, config)
# saver.save(sess, path + '/model.ckpt' , global_step=max_size)

ModuleNotFoundError: No module named 'gensim'

In [20]:
model.most_similar('범죄/Noun',topn = 20)  ## topn = len(model.wv.vocab)

/home/minwookje/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('...../Punctuation', 0.7059279084205627),
 ('맞다/Verb', 0.6767151355743408),
 ('점/Noun', 0.6739176511764526),
 ('인하다/Adjective', 0.6714572906494141),
 ('인간/Noun', 0.670412003993988),
 ('계획/Noun', 0.6638398170471191),
 ('년/Noun', 0.6505692005157471),
 ('정의/Noun', 0.6504296660423279),
 ('여기/Noun', 0.6488509774208069),
 ('부분/Noun', 0.6485463380813599),
 ('클럽/Noun', 0.6478688716888428),
 ('..../Punctuation', 0.6448889970779419),
 ('남성/Noun', 0.6372814774513245),
 ('사망/Noun', 0.6371808052062988),
 ('살해/Noun', 0.6371168494224548),
 ('사진/Noun', 0.636451780796051),
 ('16/Number', 0.6353298425674438),
 ('아시아/Noun', 0.6316608190536499),
 ('불구/Noun', 0.6313873529434204),
 ('u/Alpha', 0.631290078163147)]

In [ ]:
from keras.preprocessing.text import *
from keras.preprocessing.sequence import skip